2023-11-26 16:12:41.048613: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ModuleNotFoundError: No module named 'pandas'

### Loading data

In [ ]:
# Assuming 'monks1_train_path' is the path to your data file
monks1_train_path = os.getcwd() + '/../../data/monks/monks-1.train'

# Specify the column names or let pandas infer them
column_names = ['target', 'col1', 'col2', 'col3', 'col4', 'col5', 'col6', 'id']

# Load the data into a pandas DataFrame
df = pd.read_csv(monks1_train_path, names=column_names, delimiter=' ')

# Display the loaded data
df

In [ ]:
y = df["target"].values